# ImbalanceMetrics (Regression): Usage
## Example 2: Advanced

## Dependencies
First, we load the required dependencies. Here we import regression_metrics from imbalance_metrics to evalute the result we get from the HistGradientBoostingClassifier. In addition, we use pandas,numpy and LabelEncoder for data handling, and train_test_split to split the dataset.

In [5]:
## load dependencies
from imbalance_metrics import regression_metrics as rm
import pandas as pd 
from sklearn.ensemble import HistGradientBoostingClassifier
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error , r2_score
from sklearn.model_selection import train_test_split



## Data
After, we load our data. In this example, we use the Ames Housing Dataset training split retreived from Kaggle, originally complied by Dean De Cock. 
Link to original dataset - https://www.kaggle.com/datasets/prevek18/ames-housing-dataset

In [11]:
## load data
df = pd.read_csv(
    'https://raw.githubusercontent.com/paobranco/ImbalanceMetrics/main/data/housing(processed).csv', index_col=None, na_values=['NA']
)
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Data preprocessing

In [ ]:
# We need to drop rows with nan values
df=df.drop(columns=["Alley","PoolQC","Fence","MiscFeature"])

# We need to change the data type from object to number
object_data = df.select_dtypes(include='object')
num_data = df.select_dtypes(exclude='object')

enc = LabelEncoder()
for i in range(0, object_data.shape[1]):
    object_data.iloc[:,i] = enc.fit_transform(object_data.iloc[:,i])   

In [4]:
# Merging the numerical part of the dataset and processed obeject part of the dataset together.
df = pd.concat([num_data, object_data], axis = 1)
df=df.fillna(0)

In [5]:
# Show the description of the dataset
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,57.623288,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.117123,443.639726,...,2.339726,5.749315,3.804795,2.485616,1.284247,3.927397,3.960959,1.856164,7.513014,3.770548
std,421.610009,42.300571,34.664304,9981.264932,1.382997,1.112799,30.202904,20.645407,180.731373,456.098091,...,0.830161,0.979659,1.398954,1.933206,0.892831,0.647822,0.566832,0.496592,1.552100,1.100854
min,1.000000,20.000000,0.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,365.750000,20.000000,42.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,2.000000,6.000000,2.000000,1.000000,1.000000,4.000000,4.000000,2.000000,8.000000,4.000000
50%,730.500000,50.000000,63.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,3.000000,6.000000,4.000000,1.000000,1.000000,4.000000,4.000000,2.000000,8.000000,4.000000
75%,1095.250000,70.000000,79.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,164.250000,712.250000,...,3.000000,6.000000,5.000000,5.000000,2.000000,4.000000,4.000000,2.000000,8.000000,4.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,3.000000,6.000000,5.000000,6.000000,3.000000,5.000000,5.000000,2.000000,8.000000,5.000000


In [6]:
# Assign x and y values from the dataframe as train and test.
X = df.drop(columns="SalePrice") # Every other columns except price      
y = df["SalePrice"]    # y = price


In [7]:
#Split X and y into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

## Model
After, we train our model with data. In this example, we use the `HistGradientBoostingClassifier()` from sklearn. This model will predict 'SalePrice' as y_pred which we will compare with true value y in our evaluation.

In [8]:
reg = HistGradientBoostingClassifier().fit(X_train, y_train)
y_pred = reg.predict(X_test)

## phi relevance arguments
    

In this example, we dive a bit deeper. In beginer example, we left all the default values for `phi` relevance function. In this example, we are going pass few arguments to control the `phi` relevance. The arguments are given below :

* The `rel_method` argument takes a string, either `'auto'` or `'manual'`. It specifies how relevant or rare "minority" values in `y` are determined. If `'auto'` is specified, "minority" values are automatically determined by box plot extremes. If `'manual'` is specified, "minority" values are determined by the user. In this example, we input `'auto'` with the following input: `rel_method = 'auto'`

* The `rel_xtrm_type` argument takes a string, either `'low'` or `'both'` or `'high'`. It indicates which region of the response variable `y` should be considered rare or a "minority", when `rel_method = 'auto'`. In this example, we input `'high'` (default is `'both'`) in the interest of over-sampling high "minority" values in `y` with the following input: `rel_xtrm_type = 'high'`

* The `rel_coef` argument takes a positive real number. It specifies the box plot coefficient used to automatically determine extreme and therefore rare "minority" values in `y`, when `rel_method = 'auto'`. In this example, we input `2.25` (default is `1.50`) to increase the box plot extremes with the following input: `rel_coef = 2.25`

In [9]:
rel_method = 'manual'      ## string ('auto' or 'manual')
rel_xtrm_type = 'low'   ## string ('low' or 'both' or 'high')
rel_coef = 2.25           ## positive real number (0 < R)


`rg_mtrx` is the "ctrl_pts" array indicating the regions of interest in the response variable y and their corresponding relevance values mapped to either 0 or 1, expressed as [y, 0, 1]

In [10]:

## specify phi relevance values
rg_mtrx = [

    [35000,  1, 0],  ## over-sample ("minority")
    [125000, 0, 0],  ## under-sample ("majority")
    [200000, 0, 0],  ## under-sample
    [250000, 0, 0],  ## under-sample
]

## Evaluation

In this section, we present the evaluation metrics available in the package. These metrics can be used to evaluate the performance of models while the dataset is imbalanced. In this example, we are only going to use regression metrics. 
This package includes the following evaluation metrics for regression:

* `phi_weighted_r2` : Calculates the R^2 score between 'y' and 'y_pred' with weighting by `phi`.

* `phi_weighted_mse` : Calculates the mean squared error between 'y' and 'y_pred' with weighting by `phi`.

* `phi_weighted_mae` : Calculates the mean absolute error between 'y' and 'y_pred' with weighting by `phi`.

* `phi_weighted_root_mse` : Calculates the root mean squared error between 'y' and 'y_pred' with weighting by `phi`.

* `ser_t` : Calculates the Squared error-relevance values between 'y' and 'y_pred' with weighting by `phi` at thershold 't'.

* `sera` : Calculates the Squared error-relevance areas (SERA) between 'y' and 'y_pred'.


But first, we are going to import `mean_squared_error`, `mean_absolute_error` and `r2` from sklearn package to see their evaluation. 

In [11]:
# Calculate mean_squared_error
mse = mean_squared_error(y_test , y_pred )
print("Mean Squared Error:", mse)

# Calculate mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae) 

# Calculate r2_score
r2 = r2_score(y_test, y_pred)
print("R2:", r2)            

Mean Squared Error: 3069451226.6950207
Mean Absolute Error: 35189.84854771784
R2: 0.44421870645793937


In next part, we have used the metrics functions from our package, "ImbalanceMetrics".

In [12]:
# Calculate phi_weighted_mse. Here the weight calculated by phi relevance function is done by passig rel_method,rel_xtrm_type, rel_coef and ctrl_pts.
wmse = rm.phi_weighted_mse (y_test , y_pred, method = rel_method, xtrm_type = rel_xtrm_type, coef = rel_coef, ctrl_pts = rg_mtrx)
print("Weighted Mean Squared Error:", wmse)

# Calculate phi_weighted_mae. Here the weight calculated by phi relevance function is done by passig rel_method,rel_xtrm_type, rel_coef and ctrl_pts.
wmae = rm.phi_weighted_mae (y_test , y_pred, method = rel_method, xtrm_type = rel_xtrm_type, coef = rel_coef, ctrl_pts = rg_mtrx)
print("Weighted Mean Absolute Error:", wmae)

# Calculate phi_weighted_r2. Here the weight calculated by phi relevance function is done by passig rel_method,rel_xtrm_type, rel_coef and ctrl_pts.
wr2 = rm.phi_weighted_r2 (y_test , y_pred, method = rel_method, xtrm_type = rel_xtrm_type, coef = rel_coef, ctrl_pts = rg_mtrx)
print("Weighted R2:", wr2)

Weighted Mean Squared Error: 3568072516.842982
Weighted Mean Absolute Error: 39348.904251551736
Weighted R2: -9.256115579681959


In [13]:
# Calculate phi_weighted_root_mse. Here the weight calculated by phi relevance function is done by passig rel_method,rel_xtrm_type, rel_coef and ctrl_pts.
wrmse = rm.phi_weighted_root_mse (y_test , y_pred, method = rel_method, xtrm_type = rel_xtrm_type, coef = rel_coef, ctrl_pts = rg_mtrx)
print("Weighted Root Mean Squared Error:", wrmse)

Weighted Root Mean Squared Error: 59733.345100061


In [15]:
# Calculate ser_t. Here the weight calculated by phi relevance function is done by passig rel_method,rel_xtrm_type, rel_coef and ctrl_pts. The threshold is defined by the user. In this case threshold is .055.
threshold = .055
ser_t= rm.ser_t(y_test, y_pred,threshold, method = rel_method,  xtrm_type = rel_xtrm_type, ctrl_pts = rg_mtrx)
print("Ser("+str(threshold)+"):", ser_t)

# Calculate ser_t. Here the weight calculated by phi relevance function is done by passig rel_method,rel_xtrm_type, rel_coef and ctrl_pts.. The threshold is defined by the user. In this case threshold is .3.
threshold = .3
ser_t= rm.ser_t(y_test, y_pred,threshold, method = rel_method,  xtrm_type = rel_xtrm_type, ctrl_pts = rg_mtrx)
print("Ser("+str(threshold)+"):", ser_t)

# Calculate sera. Here the weight calculated by phi relevance function is done by passig rel_method,rel_xtrm_type, rel_coef and ctrl_pts. Here the the relevance factor, rel is default value (None)
sera= rm.sera(y_test, y_pred, method = rel_method,  xtrm_type = rel_xtrm_type, ctrl_pts = rg_mtrx)
print("Sera:", sera)


Ser(0.055): 146227943049.0
Ser(0.3): 92713950549.0
Sera: 90660030998.54498


## Optional
Users can also calculate their own weights by using `phi` relevance function. Here rel_method,rel_xtrm_type and rel_coef are passed as arguments.

In [16]:
# Calculate weight using phi relevance function.
rm.calculate_phi(y_test, method = rel_method,  xtrm_type = rel_xtrm_type, coef = rel_coef , ctrl_pts = rg_mtrx)

[0.00036762688614522254,
 0.027999999999999692,
 0.0,
 0.0,
 0.0,
 0.0,
 0.010747256515774684,
 0.0,
 0.07407407407407385,
 0,
 0,
 0.26669718792866925,
 0.0,
 0,
 0.01289048994607378,
 0.0,
 0.0,
 0.027999999999999692,
 0.0,
 0.0,
 0.0,
 0.0,
 0.09872256515775013,
 0.0,
 0.5747499999999999,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.07407407407407385,
 0.0,
 0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0,
 0.0,
 0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.00036762688614522254,
 0.0,
 0.06506447187928655,
 0.056565157750342765,
 0.0,
 0.0628911608367626,
 0.0,
 0.0,
 0.0,
 0,
 0.0,
 0.0,
 0.007249999999999646,
 0.0,
 0.5166639231824417,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6890510397805212,
 0.0,
 0,
 0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0,
 0,
 0.0,
 0.0,
 0.00325925925925874,
 0,
 3.7009602192883406e-06,
 0.0,
 0.0,
 0.41700960219478733,
 0.0,
 0,
 0.0,
 0.0,
 0.00325925925925874,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0

## Conclusion

In this package, we have presented a set of evaluation metrics specifically designed for imbalanced domains. Our package, "ImbalanceMetrics", provides a comprehensive set of evaluation metrics to assess the performance of machine learning models trained on imbalanced datasets.

Our package includes several evaluation metrics that address the challenges of imbalanced domains. These metrics can provide a more accurate assessment of model performance than traditional metrics, which can be misleading in imbalanced domains.

To learn more about our package, please refer to the documentation, which includes detailed descriptions of all the available metrics and their usage.

